In [147]:
import pandas as pd

df = pd.read_csv('./data/cleaned/consumption_temp_cleaned.csv')
#df = pd.read_csv('./data/cleaned/wo_helsingfors/consumption_temp_cleaned.csv')

print(df.head(10))


                  time   location  consumption  temperature
0  2022-04-07 21:00:00     bergen     1.113325         -0.3
1  2022-04-07 21:00:00       oslo     4.092830          1.0
2  2022-04-07 21:00:00  stavanger     2.057858          1.3
3  2022-04-07 21:00:00     tromsø     1.246582         -3.9
4  2022-04-07 21:00:00  trondheim     1.970098         -2.8
5  2022-04-07 22:00:00     bergen     1.050327          0.0
6  2022-04-07 22:00:00       oslo     3.818095          0.4
7  2022-04-07 22:00:00  stavanger     1.918996          0.8
8  2022-04-07 22:00:00     tromsø     1.180321         -4.3
9  2022-04-07 22:00:00  trondheim     1.839443         -3.3


In [148]:
### Holiday binary encoding
import holidays

# Convert 'time' to datetime
df['time'] = pd.to_datetime(df['time'])

# Extract date from 'time'
df['date'] = df['time'].dt.date

# Get Norwegian holidays
no_holidays = holidays.Norway(years = [2022, 2023])

print(no_holidays)

# Create a feature for whether or not the date is a holiday
df['is_holiday'] = df['date'].isin(no_holidays)


### Other time related features

# Extracting weekday feature
df['weekday'] = df['time'].dt.weekday

# Creating features for time of day
df['hour_of_day'] = df['time'].dt.hour

# Create interaction for weekday and hour_of_day
df['weekday_hour'] = df['weekday'] * df['hour_of_day']

# Create a new feature 'is_weekend' cause consumptions are higher on weekends
df['is_weekend'] = df['weekday'].apply(lambda x: 1 if x >= 5 else 0)

### Lag feature

# Lag features for how the consumption was 7 days earlier
df['consumption_lag_last_week'] = df.sort_values('date').groupby(['hour_of_day', 'location'])['consumption'].shift(7)

# Lag features for how the consumption was 5 days earlier since we only have 
# access to data going 5 days back
df['consumption_lag_5_days'] = df.sort_values('date').groupby(['hour_of_day', 'location'])['consumption'].shift(5)


# print(df.head(10))

print(df[df['is_holiday'] == True])


{datetime.date(2022, 1, 1): 'Første nyttårsdag', datetime.date(2022, 4, 14): 'Skjærtorsdag', datetime.date(2022, 4, 15): 'Langfredag', datetime.date(2022, 4, 17): 'Første påskedag', datetime.date(2022, 4, 18): 'Andre påskedag', datetime.date(2022, 5, 1): 'Arbeidernes dag', datetime.date(2022, 5, 17): 'Grunnlovsdag', datetime.date(2022, 5, 26): 'Kristi himmelfartsdag', datetime.date(2022, 6, 5): 'Første pinsedag', datetime.date(2022, 6, 6): 'Andre pinsedag', datetime.date(2022, 12, 25): 'Første juledag', datetime.date(2022, 12, 26): 'Andre juledag', datetime.date(2023, 1, 1): 'Første nyttårsdag', datetime.date(2023, 4, 6): 'Skjærtorsdag', datetime.date(2023, 4, 7): 'Langfredag', datetime.date(2023, 4, 9): 'Første påskedag', datetime.date(2023, 4, 10): 'Andre påskedag', datetime.date(2023, 5, 1): 'Arbeidernes dag', datetime.date(2023, 5, 17): 'Grunnlovsdag', datetime.date(2023, 5, 18): 'Kristi himmelfartsdag', datetime.date(2023, 5, 28): 'Første pinsedag', datetime.date(2023, 5, 29): 'An

In [149]:
### Seasonality feature

# Assuming that:
# 1 represents spring (March, April, May),
# 2 represents summer (June, July, August),
# 3 represents fall (September, October, November), 
# 4 represents winter (December, January, February)
seasons = {1: '4', 2: '4', 3: '1', 4: '1', 5: '1', 6: '2', 7: '2', 8: '2', 9: '3', 10: '3', 11: '3', 12: '4'}

# Apply the mapping to the 'month' column
df['season'] = df['time'].dt.month.map(seasons)

# Create interaction for is_holiday == True and season == 4, since consumption
# only goes up in the winder when it is a holiday
df['holiday_season_winter'] = ((df['is_holiday'] == True) & (df['season'] == 4)).astype(int)


### Temperature difference from the previous day at the same time

# # If direction of change matters, we should keep neg/pos values (this is usually
# # the case when working with data involving temperature differences)
# # If only magnitude of change matters, we should take the absolute value
# df['temperature_diff_prev_day'] = df.sort_values('date').groupby(['hour_of_day', 'location'])['temperature'].diff()


### Moving averages for consumption over the past week

# # We apply a transform function to each location. Since each row in the data
# # is a  by hour, and each location has 24 hours of data for each day,
# # we can apply a rolling window of 168 (24*7) to get the past week's consumption
# # grouped by each location
# df['consumption_ma_week'] = df.groupby('location')['consumption'].transform(lambda x: x.rolling(24*7).mean())


### Rolling window statistics for consumption over the past week

# df['consumption_max_week'] = df.groupby(['location'])['consumption'].transform(lambda x: x.rolling(24*7).max())
# df['consumption_min_week'] = df.groupby(['location'])['consumption'].transform(lambda x: x.rolling(24*7).min())


### Exponential smoothing for consumption

# here x.ewm(alpha=0.5).mean() calculates the exponential weighted moving 
# average for consumption for each location. alpha=0.5 is the decay factor, 
# which controls the rate of decay. A large alpha will make the EWM more 
# responsive to recent values, while a small alpha will make the EWM respond 
# more to historical values. We can play around with this value.

# Forgot to play around with this one, after doing so results were improved
# seems like 0.93, weighting closer to recent data gives better result, any
# higher and it overfits and lower is slightly worse MSE
df['consumption_ewm_alpha_0.93'] = df.groupby(['location'])['consumption'].apply(lambda x: x.ewm(alpha=0.93).mean()).reset_index(level=0, drop=True)

# Save results to csv
df.to_csv('./data/cleaned/consumption_temp_w_features.csv', index=False)
#df.to_csv('./data/cleaned/wo_helsingfors/consumption_temp_w_features.csv', index=False)


In [150]:
# Apply to model by setting target = consumption, scaling, fitting model, etc...
X = df.drop('consumption', axis=1)
y = df['consumption']
# ... and so on
